# $\pi$ by means of the arithmetic-geometric mean

The mathematical constant $\pi$ can be determined by means of the arithmetic-geometric mean as demonstrated in [E. Salamin, Math. Comp. **30**, 565 (1976)](https://doi.org/10.2307%2F2005327). Theorem 1b in that paper states that
$$\pi = \frac{4[\mathrm{agm}(1, 2^{-1/2})]^2}{ 1-\sum_{n=1}^\infty 2^{n+1}c_n^2}\,.$$
To introduce the arithmetic-geometric mean $\mathrm{agm}(a, b)$ of $a$ and $b$ and the numbers $c_n$ appearing in this relation, we consider the recursion
$$a_{n+1} = \frac{a_n+b_n}{2}\,,\quad b_{n+1} = \sqrt{a_{n}b_{n}}$$
with the initial values $a_0=a, b_0=b$. For $n\rightarrow\infty$, $a_n$ and $b_n$ converge to the same number, the arithmetic-geometric mean of $a$ and $b$. Furthermore, we define
$$c_n^2=a_n^2-b_n^2\,.$$

In a first step, an implementation of this prescription using floats shall be developed. In this way, we will familiarize ourselves with the computation of $\pi$ by means of the arithmetic-geometric mean. Ideally, the iteration is carried out until $c_n$ becomes zero. However, rounding errors might prevent the series $(a_n, b_n)$ reach a point where $a_n=b_n$ within floating point precision.

In a second step, the calculation should be done by means of integers thus taking advantage of the fact that in Python integers can become sufficiently large. This approach will allow us to determine $\pi$ to a large number of digits.

## Part 1: Implementation with floats

Let us start by getting acquainted with the arithmetic-geometric mean. Implement a function which takes $a_n$ and $b_n$ as arguments and returns $a_{n+1}$, $b_{n+1}$, as well as $a_{n+1}^2-b_{n+1}^2$.

In [ ]:
# if needed, modules should be imported before functions are defined
# BEGIN SOLUTION
from math import sqrt
# END SOLUTION

def agm_iteration(a, b):
    """perform one arithmetic-geometric iteration
    
       This function takes two floats a and b as
       a_n and b_n and returns a_{n+1}, b_{n+1},
       and the difference of their squares.
    """
    # BEGIN SOLUTION
    a, b = (a+b)/2, sqrt(a*b)
    c2 = a*a-b*b
    return a, b, c2
    # END SOLUTION
    

Execute the following two cells to test your code.

In [ ]:
try:
    agm_iteration(1, 1)
except NameError as e:
    if 'sqrt' in repr(e):
        raise AssertionError('Did you forget to import a required module?')
assert agm_iteration(1, 1) is not None, 'Did you return a result?'
assert len(agm_iteration(1, 1)) == 3, 'It seems that you are not returning three values.'

In [ ]:
import random
from math import sqrt
x = random.random()
assert agm_iteration(x, x) == (x, x, 0), 'There is a problem even for the trivial case of equal arguments.'
assert agm_iteration(x, 0) == (0.5*x, 0, 0.25*x**2), 'There is a problem if one of the arguments is zero.'
y = random.random()
assert agm_iteration(x, y)[1] != sqrt(0.5*(x+y)*y), ('Make sure that you do not accidentally change the '
                                                     'variable a before you evaluate the square root.')
assert agm_iteration(x, y)[:-1] == (0.5*(x+y), sqrt(x*y)), 'There is an error in the new values of a and b.'
assert abs(agm_iteration(x, y)[2]-0.25*(x-y)**2) < 1e-14, 'The difference of the squares is not correct.'

Now, you can determine $\pi$ by means of the arithmetic-geometric mean. Since at this point floats are used, rounding errors will prevent you from obtaining the optimal result. You should aim for an relative error with respect to the correct result smaller than $2\cdot10^{-14}$.

In [ ]:
# if needed, modules should be imported before functions are defined
# BEGIN SOLUTION
from math import sqrt
# END SOLUTION

def pi_agm_float():
    """determine pi by means of the arithmetic-geometric mean
    
    """
    # BEGIN SOLUTION
    a = 1
    b = 1/sqrt(2)
    c_sum = 0
    faktor_two = 2
    c2 = 1

    while c2 > 1e-15:
        a, b = (a+b)/2, sqrt(a*b)
        c2 = a*a-b*b
        faktor_two = faktor_two*2
        c_sum = c_sum+faktor_two*c2

    return 4*a**2/(1-c_sum)
    # END SOLUTION

By executing the following two cells, you can verify that your code runs correctly.

In [ ]:
try:
    result = pi_agm_float()
except NameError as e:
    if 'sqrt' in repr(e):
        raise AssertionError('Did you forget to import a required module?')
assert result is not None, 'Did you return a result?'
assert type(result) == type(1.), 'The result should be a float.'

In [ ]:
from math import pi
result = pi_agm_float()
relerror = abs(result-pi)/pi
assert relerror <= 2e-14, 'The relative error {:.3g} is larger than allowed.'.format(relerror)
assert relerror != 0., 'Did you simply return the value of π from the math module?'

## Part 2: Implementation with integers

A much higher precision can be achieved by doing the complete calculation in integer arithmetic after expanding the fraction by a more or less large power of ten.

As a first step, we need to be able to determine the square root of a large integer. Explain why the `sqrt`-function of the `math`-module cannot be used to take the square root of a large integer. Enter your answer in the cell below. *Hint*: If you do not know what could be going wrong, try to apply the `sqrt`-function to a really large integer.

`sqrt` will first try to convert the integer argument into a float. This attempt will fail if the integer is larger than the largest possible float.

A straightforward way to take the square root of an arbitrary integer is by means of [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method). Implement the corresponding algorithm.

In [ ]:
def int_sqrt(n2, n0):
    """determine square root of the integer n2
    
       Newton's method is used to determine the square root
       of n2 starting from the initial integer n0. An initial
       value n0 smaller than 1 will be replaced by 1 in order
       to avoid a possible division by zero.
    """
    ### BEGIN SOLUTION
    n = max(n0, 1)
    n_old = 0
    while n != n_old:
        n_old = n
        n = (n*n+n2)//(2*n)
    return n
    ### END SOLUTION

Check the correctness of your implementation by integers of your own choice and by executing the following two cells.

In [ ]:
result = int_sqrt(4, 1)
assert result is not None, 'Did you return a result?'
assert type(result) == type(1), 'The result should be an integer.'

In [ ]:
from random import randrange
n = randrange(10**800, 10**801)
assert int_sqrt(n*n, 1) == n, 'The square root of\n{}\nis\n{}\nbut a different result was found.'.format(n*n, n)
n = 2*10**800
result = int_sqrt(n, 1)
assert abs(result**2-n) <= 2*result+1, 'The integer square root of 2*10**200 is not found correctly.'

In order to reduce the compute time, the choice of a good initial value `n0` is important. For this purpose, the `sqrt`-function from the `math`-module can be useful. In order to be able to handle even very large integers, approximate the integer by an integer containing the first 12 or 13 digits multiplied with an even power of 10. The square root of the first factor can then be determined by means of the `sqrt`-function while the square root of the second factor is obtained trivially.

Implementing this idea requires to determine the number of digits of an integer and to determine its first 12 or 13 digits. To this end, it is useful to first convert the integer into a string. If you need help, take a look at [Integer as String](~IntegerAsString.ipynb).

In [ ]:
# imports should come before the function definitions
### BEGIN SOLUTION
from math import sqrt
### END SOLUTION

def sqrt_initial(n2):
    """determine an approximate value for the square root of n2
    
       This function uses math.sqrt on the first 12 or 13 digits
       of n2 for an even or odd number of digits of n2, respectively,
       to obtain an integer approximation for the square root of n2.
       The result can serve as an input for determining the square root
       of an integer by means of Newton's method.
    
    """
    ### BEGIN SOLUTION
    n2_str = str(n2)
    nrdigits = len(n2_str)
    keepdigits = 12 + (nrdigits % 2)
    if keepdigits > nrdigits:
        return int(sqrt(n2))
    leadingdigits = n2_str[:keepdigits]
    lead_sqrt_estimate = sqrt(float(leadingdigits))
    return int(lead_sqrt_estimate)*10**((nrdigits-keepdigits)//2)
    ### END SOLUTION

In [ ]:
n2 = 12345678901234567890
result = sqrt_initial(n2)
assert result is not None, 'Did you return a result?'
assert isinstance(result, int), 'The result should be an integer.'

In [ ]:
from random import randrange
for exponent in range(1, 801):
    for _ in range(10):
        n = randrange(10**exponent, 10**(exponent+1))
        result = sqrt_initial(n*n)
        assert abs(n-result) < max(1, result//10**5), ('The estimate is too bad.\n'
            'expected value: {}\nestimate: {}'.format(n, result))

In [ ]:
assert sqrt_initial(200) == 14, 'There seems to be a problem for small arguments like n2=200.'

Now, we are in a position to replace the implementation of the arithmetic-geometric mean for floats by a corresponding version for integers.

In [ ]:
def int_agm_iteration(a, b):
    """perform one arithmetic-geometric iteration for
       a pair of integers
    
       This function takes two integers a and b as
       a_n and b_n and returns a_{n+1}, b_{n+1},
       and the difference of their squares.
    """
    # BEGIN SOLUTION
    a, b = (a+b)//2, int_sqrt(a*b, sqrt_initial(a*b))
    c2 = a*a-b*b
    return a, b, c2
    # END SOLUTION

In [ ]:
result = int_agm_iteration(1, 1)
assert result is not None, 'Did you return a result?'
assert len(result) == 3, 'It seems that you are not returning three values.'
assert all(type(x) == type(1) for x in result), 'It seems that not all of your return values are integers.'

In [ ]:
import random
x = random.randrange(10**50, 10**52)
assert int_agm_iteration(x, x) == (x, x, 0), 'There is a problem even for the trivial case of equal arguments.'
y = random.randrange(10**50, 10**52)
result = int_agm_iteration(x, y)
assert result[0] == (x+y)//2, 'The arithmetic mean of a and b is not correct.'
assert abs(result[1]**2-x*y) < 2*x*y+1, 'The geometric mean of a and b is not correct.'
assert result[0]**2-result[1]**2 == result[2], 'The difference of the squares of a and b is not correct.'

With the functions developed so far, write a function which determines $\pi$ to a large number of digits, for example 1000 or even more digits. It is sufficient, if you produce an integer starting with `314...`. There is no need to generate an output with a decimal comma simulating a float result.

In [ ]:
def pi_agm_int(scale):
    """determine pi by means of the arithmetic-geometric mean
       using integer arithmetic after expanding the fraction by
       scale
       
       scale: numerator and denominator are expanded by the
              square of this parameter, typically a power of ten
    """
    ### BEGIN SOLUTION
    a = scale
    halfscaled = scale**2//2
    b = int_sqrt(halfscaled, sqrt_initial(halfscaled))
    denominator = scale**2
    factor_two = 2
    while a != b:
        a, b, c2 = int_agm_iteration(a, b)
        factor_two = 2*factor_two
        denominator = denominator-factor_two*c2
    numerator = 4*a**2
    return numerator*scale//denominator
    ### END SOLUTION   

In [ ]:
result = pi_agm_int(10**10)
assert result is not None, 'Did you return a result'
assert type(result) == type(1), 'The result should be an integer'

In [ ]:
result = str(pi_agm_int(10**1000))
assert result.startswith('3'), 'The result does not start with 3.'
assert result[92:94] == '42', 'The digits at positions 92 and 93 after the comma are incorrect.'
assert result[854:858] == '1000', 'The digits at positions 854 to 857 after the comma are incorrect.'

Suggestion for further work:
* Use the result returned by `pi_agm_int` to produce a nice output of the form   
  π=3.141... (continuing with the digits which you have obtained)
* As an alternative, write a function which divides two large integers and produces $\pi$ as a float number digit by digit.
* Write a function to search for digit patterns in $\pi$.